## NEW
- skip threading and FastRelax

In [2]:
import os
num_gpus=3 #How many gpus to use

scaffold_dir="scaffolds/CD4-dom2-winners/"
separate_scaffold_folders=True

prefix="CD4-dom1-sample4"
target_pdb="target_pdb/cd4-dom1-2-noHOH-1cdh.pdb"
hotspots='ppi.hotspot_res=[A34,A35,A43]'

num_of_diffusions=10 # Number of RF diffusions to do per scaffold
num_seqs=50 # How many MPNN sequences to generate per RF diffusion
num_filtered_seqs=50 # How many of the MPNN generated sequences per RF diffusion to keep for AF2

# Other parameters
num_recycles=3 # AF2 recycles
sampling_temp=0.0001 # ProteinMPNN sampling temperature

def get_subdirectories(folder_path):
    subdirectories = []
    for entry in os.scandir(folder_path):
        if entry.is_dir():
            subdirectories.append(entry.path)
    return subdirectories

if separate_scaffold_folders:
    scaffold_list=get_subdirectories(scaffold_dir)
    scaffolds=len(scaffold_list)
    print(scaffold_list)
print(f"JOB: {prefix}")
print(f"TARGET: {target_pdb}")
print(f"HOTSPOTS: {hotspots}")

def directory_exists(directory):
    return os.path.isdir(directory)

directory = f"output/{prefix}"
if directory_exists(directory):
    print("""\033[1;31;40m#####################
PREFIX ALREADY EXISTS
#####################""")
else:
    print("Prefix does not exist yet")
print("\033[0mNumber of gpus already in use: ", end="")
!squeue | grep -c "gpu"

['scaffolds/CD4-dom2-winners/51_3H', 'scaffolds/CD4-dom2-winners/lcb3']
JOB: CD4-dom1-sample4
TARGET: target_pdb/cd4-dom1-2-noHOH-1cdh.pdb
HOTSPOTS: ppi.hotspot_res=[A34,A35,A43]
Prefix does not exist yet
Number of gpus already in use: 43


## Runs slurm array script

In [3]:
import subprocess
os.makedirs(directory, exist_ok=True)
if separate_scaffold_folders:
    if num_gpus%scaffolds==0:   #If number of gpus is divisible by the number of scaffolds, make it so that each scaffold has its own gpu(s)
        gpus_per_scaffold = num_gpus//scaffolds
        diffusions_per_gpu = num_of_diffusions//gpus_per_scaffold
        for i in range(scaffolds):
            # Setup arguments
            scaffold_dir=scaffold_list[i]
            script_arguments=f"{diffusions_per_gpu} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {hotspots}"
            bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{gpus_per_scaffold}"
            
            command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

            # Run the array bash script
            print(f"Running {command}")
            subprocess.run(command, shell=True)
    else:
        num_per_gpu=num_of_diffusions//num_gpus
        scaffold_string = ','.join(scaffold_list)   #Converts list of scaffold directories into a string for parsing in shell script
        script_arguments=f"{num_per_gpu} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_string} {prefix} {target_pdb} {hotspots}"
        bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{num_gpus}"
            
        command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

        # Run the array bash script
        print(f"Running {command}")
        subprocess.run(command, shell=True)

else:
    script_arguments=f"{num_of_diffusions} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {hotspots}"
    bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{num_gpus}"
    
    command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

    # Run the array bash script
    print(f"Running {command}")
    subprocess.run(command, shell=True)

#print(f"command: {command}")

Running sbatch --job-name=CD4-dom1-sample4_binder_docking --array=1-3 helper_scripts/binder_dock_nothread.sh 3 50 50 3 0.0001 scaffolds/CD4-dom2-winners/51_3H,scaffolds/CD4-dom2-winners/lcb3 CD4-dom1-sample4 target_pdb/cd4-dom1-2-noHOH-1cdh.pdb ppi.hotspot_res=[A34,A35,A43]
Submitted batch job 389203


In [6]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          389186_1       gpu CD4-dom2  lhafner  R    1:29:49      1 compute-6-1
          389187_1       gpu CD4-dom2  lhafner  R    1:29:49      1 compute-6-1
          389203_1       gpu CD4-dom1  lhafner  R       9:29      1 compute-6-2
          389203_2       gpu CD4-dom1  lhafner  R       9:29      1 compute-6-4
          389203_3       gpu CD4-dom1  lhafner  R       9:29      1 compute-6-4


In [21]:
###############
### CAUTION ###
###############
##########################################
### DO NOT RUN IF SLURM JOBS ARE RUNNING #
##########################################
# DELETES ALL slurm.out files. 
import os

directory = os.getcwd()

for filename in os.listdir(directory):
    if filename.startswith("slurm") and filename.endswith(".out"):
        try:
            os.remove(os.path.join(directory, filename))
            print(f"{filename} deleted.")
        except Exception as e:
            print(f"Failed to delete {filename}. Error: {e}")

slurm-388518_2.out deleted.
slurm-388516_2.out deleted.
slurm-388710_1.out deleted.
slurm-388779_2.out deleted.
slurm-388707_1.out deleted.
slurm-388514_1.out deleted.
slurm-388671_2.out deleted.
slurm-388709_1.out deleted.
slurm-388681_2.out deleted.
slurm-388516_1.out deleted.
slurm-388667_2.out deleted.
slurm-388778_2.out deleted.
slurm-388778_1.out deleted.
slurm-388690_1.out deleted.
slurm-388780_1.out deleted.
slurm-388545_2.out deleted.
slurm-388686_2.out deleted.
slurm-388708_1.out deleted.
slurm-388779_1.out deleted.
slurm-388781_2.out deleted.
slurm-388517_1.out deleted.
slurm-388781_1.out deleted.
slurm-388780_2.out deleted.
slurm-388681_1.out deleted.
slurm-388686_1.out deleted.
slurm-388517_2.out deleted.
slurm-388782_1.out deleted.
slurm-388669_2.out deleted.
slurm-388706_1.out deleted.
slurm-388669_1.out deleted.
slurm-388514_2.out deleted.
slurm-388518_1.out deleted.
slurm-388688_2.out deleted.
slurm-388667_1.out deleted.
slurm-388515_1.out deleted.
slurm-388515_2.out d